In [2]:
import numpy as np

#PageRank com autodecomposições
###Grupo: Alberto Silva, Daniel Santos, Marlon Costa

##O que é PageRank?

O PageRank foi desenvolvido por volta de 1995 na Universidade de Stanford por Larry Page, daí vem o nome “Page” Rank.Foi o primeiro algoritmo utilizado pelo Google para classificar páginas da internet.Tem como objetivo avaliar a relevância de uma determinada página.A idéia por trás do PageRank se baseia em uma rede citações podendo ser representada por um grafo, onde cada nó representa uma página Web e a ligação entre as páginas se dá por meio de uma referência que uma página faz a outra. 

![Grafo de Representação da Rede](https://www.gdax.com.br/wp-content/uploads/2016/08/PageRank-Exemplo.png)


Uma característica interessante nesse algoritmo é o valor de PageRank de um determinado Website não se dá apenas pelo número de citações mas sim pela relevância que eles possuem. 




##PageRank em termos de álgebra linear:
<br>
PageRank cria um vetor de ranks: um elemento para cada página. Também cria uma matriz de links: cada link de uma página para a outra coloca um '1' na célula apropriada na matriz.
<br><br>
Então ele pega este vetor -- inicialmente definido para algum valor padrão -- e repetidamente aplica a matriz de links a ele.

###Implementação do PageRank

In [3]:
def pagerank(M, eps=1.0e-8, d=0.85):
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    last_v = np.ones((N, 1), dtype=np.float32) * 100
    M_hat = (d * M) + (((1 - d) / N) * np.ones((N, N), dtype=np.float32))
    
    while np.linalg.norm(v - last_v, 2) > eps:
        last_v = v
        v = np.matmul(M_hat, v)
    return v


M = np.array([[0, 0, 0, 0, 1],
              [0.5, 0, 0, 0, 0],
              [0.5, 0, 0, 0, 0],
              [0, 1, 0.5, 0, 0],
              [0, 0, 0.5, 1, 0]])

v = pagerank(M, 0.001, 0.85)
v


array([[0.25404915],
       [0.13825267],
       [0.13825267],
       [0.20603414],
       [0.26341127]])

##O Poder dos Autovetores

Uma aplicação dos autovetores é a diagonalização de uma matriz. Dados os autovalores e autovetores de uma matriz, você pode criar três matrizes que quando combinadas, correspondem à matriz original. A matriz do meio lida com o escalonamento e é uma matriz diagonal, tendo os autovalores da matriz original como entradas.

Isso nos permite algo incrível: exponenciar a matriz original é equivalente a exponenciar *apenas* a matriz do meio do trio de matrizes vindas da diagonalização. E por fim, exponenciar uma matriz diagonal é equivalente à exponenciar *apenas* as entradas da diagonal principal.

##SVD

Em álgebra linear, a decomposição em valores singulares ou singular value decomposition (SVD) é a fatoração de uma matriz real ou complexa, com diversas aplicações importantes em processamento de sinais e estatística.

Formalmente, a decomposição em valores singulares de uma matriz m×n real ou complexa M é uma fatoração ou fatorização na forma:

\begin{equation}{\displaystyle M=U\Sigma V^{*},}\end{equation}

onde U é uma matriz unitária m×m real ou complexa, Σ é uma matriz retangular diagonal m×n com números reais não-negativos na diagonal, e V* (a conjugada transposta de V) é uma matriz unitária n×n real ou complexa. As entradas diagonais Σi,i de Σ são os chamados valores singulares de M. As m colunas de U e as n colunas de V são os chamados vetores singulares à esquerda e vetores singulares à direita de M, respetivamente.

A decomposição em valores singulares e a decomposição em autovalores são intimamente relacionadas. Mais especificamente:

Os vetores singulares à esquerda de M são autovetores de \begin{equation}{\displaystyle MM^{*}.} \end{equation}
Os vetores singulares à direita de M são autovetores de \begin{equation}{\displaystyle M^{*}M.} \end{equation}
Os valores singulares não-nulos de M (ao longo da diagonal de Σ) são as raízes quadradas dos autovalores não-nulos de \begin{equation}{\displaystyle M^{*}M}\end{equation}  ou \begin{equation}{\displaystyle MM^{*}.} \end{equation}

##Método das potências<br>

Em matemática, o método das potências é um algoritmo para calcular autovalores: dada uma matriz A, o algoritmo irá produzir um número λ (o autovalor) e um vetor v não nulo (o autovetor), tal que Av = λv. O algoritmo também é conhecido como a iteração de Von Mises.

O método da potência é um algoritmo muito simples. Ele não computa a decomposição matricial, e portanto pode ser usada quando A é uma grande matriz esparsa. No entanto, ele irá encontrar apenas um autovalor (aquele com o maior módulo) e poderá convergir lentamente.

In [4]:
"""https://en.wikipedia.org/wiki/Power_iteration"""


def power_iteration(A, num_simulations = 100):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k


pi = power_iteration(M)
pi


array([0.57089037, 0.28566422, 0.28566422, 0.42907961, 0.57163518])

##Algoritmo QR:
<br>
Em álgebra linear, o algoritmo QR é um algoritmo de autovalor: isso é, um procedimento para calcular os autovalores e autovetores de uma matriz. O algoritmo QR foi desenvolvido no fim dos anos 50 por John G. F. Francis e Vera N Kublanovskaya, trabalhando independentemente. A ideia básica é realizar a decomposição QR, escrevendo a matriz como um produto entre uma matriz ortogonal e uma matriz triangular superior, multiplicar os fatores em ordem reversa e iterar.<br>


In [5]:
def pure_qr(A, max_iter=50000):
    Ak = np.copy(A)
    n = A.shape[0]
    QQ = np.eye(n)
    for k in range(max_iter):
        Q, R = np.linalg.qr(Ak)
        Ak = R @ Q
        QQ = QQ @ Q
        if k % 100 == 0:
            print(Ak)
            print("\n")
    return Ak, QQ


pqr = pure_qr(M, 100)
pqr

[[ 0.          0.          0.         -0.5        -0.5       ]
 [ 1.06066017  0.          0.          0.25       -0.25      ]
 [ 0.35355339  1.          0.         -0.25        0.25      ]
 [ 0.          0.         -0.70710678  0.          0.        ]
 [ 0.          0.         -0.70710678  0.          0.        ]]




(array([[ 1.00021207e+00, -2.20147353e-01,  2.97012795e-01,
         -1.66126806e-01, -5.01313948e-02],
        [-2.67961353e-03, -9.03426848e-02, -1.09205843e+00,
         -7.96601910e-02, -8.30052562e-02],
        [-7.35530741e-04,  7.98675772e-01, -5.41980600e-02,
         -7.20796483e-02,  3.15560811e-01],
        [ 0.00000000e+00,  0.00000000e+00,  2.67032151e-04,
         -8.55671320e-01,  3.75524094e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  3.92523115e-17]]),
 array([[ 5.71757250e-01,  6.66020875e-01, -2.32466307e-01,
          4.18890507e-01, -8.86511593e-17],
        [ 2.86621357e-01,  1.58795689e-01,  5.13919934e-01,
         -3.58494669e-01,  7.07106781e-01],
        [ 2.86621357e-01,  1.58795689e-01,  5.13919934e-01,
         -3.58494669e-01, -7.07106781e-01],
        [ 4.28471810e-01, -6.06480484e-01,  3.47737347e-01,
          5.72426474e-01, -2.05432527e-33],
        [ 5.70264807e-01, -3.71706166e-01, -5.44803411e-01,
  

##Abordagem em duas etapas
Na abordagem em duas etapas para encontrar autovalores, seguimos as etapas:<br><br>
1 - Reduzir a matriz para a forma de *Hessenberg* (zeros abaixo da primeira subdiagonal).<br>
2 - Processo iterativo que faz Hessenberg convergir para uma matriz triangular. Os autovalores de uma matriz triangular são os valores da diagonal.
<br><br>
Na etapa 1, alcançamos a solução exata em um número finito de etapas, enquanto a fase 2 teoricamente nos dá a solução exata.
<br><br>
A etapa 2 é o próprio algoritmo QR. Lembrando que seria possível usar apenas o algoritmo QR, porém ele é extermamente lento.
<br><br>
Para a etapa 1, podemos usar o algoritmo de iteração de Arnoldi:

In [7]:
def arnoldi(A):
    m, n = A.shape
    assert (n <= m)

    # Hessenberg matrix
    H = np.zeros([n + 1, n])  # , dtype=np.float64)
    # Orthonormal columns
    Q = np.zeros([m, n + 1])  # , dtype=np.float64)
    # 1st col of Q is a random column with unit norm
    b = np.random.rand(m)
    Q[:, 0] = b / np.linalg.norm(b)
    for j in range(n):
        v = A @ Q[:, j]
        for i in range(j + 1):
            # This comes from the formula for projection of v onto q.
            # Since columns q are orthonormal, q dot q = 1
            H[i, j] = np.dot(Q[:, i], v)
            v = v - (H[i, j] * Q[:, i])
        H[j + 1, j] = np.linalg.norm(v)
        Q[:, j + 1] = v / H[j + 1, j]

        # printing this to see convergence, would be slow to use in practice
        # print(np.linalg.norm(A @ Q[:, :-1] - Q @ H))
    return Q[:, :-1], H[:-1, :]


Q0, H0 = arnoldi(M)
Q0, H0


(array([[ 0.00843375,  0.71690978,  0.57277896, -0.02257492,  0.39671636],
        [ 0.13770129, -0.12193278,  0.4739277 ,  0.74941542, -0.42419313],
        [ 0.34026056, -0.308898  ,  0.55697483, -0.62856354, -0.28894928],
        [ 0.71796212, -0.28545861, -0.06074172,  0.19946171,  0.59964093],
        [ 0.59137161,  0.5424649 , -0.36524824, -0.05467906, -0.46863139]]),
 array([[ 0.75320702, -0.28267775,  0.802777  ,  0.23865203, -0.04825417],
        [ 0.81602911,  0.07472611, -0.48189942, -0.22083547, -0.01218239],
        [ 0.        ,  0.85770792, -0.03920109, -0.02744823, -0.19564107],
        [ 0.        ,  0.        ,  0.18102803,  0.09294102, -0.10376424],
        [ 0.        ,  0.        ,  0.        ,  0.30108972, -0.88167306]]))

##Aplicando ao PageRank
<br>
No PageRank aplicamos várias vezes a matriz de links ao vetor PageRank. Isso equivale à repetidamente multiplicar a matriz de links por si mesma e depois de suficientes auto-multiplicações, aplicar o resultado ao vetor PageRank.
<br><br>Mas, multiplicação repetida de algo por si mesmo é a exponenciação. Então, nós podemos fazer isso exponenciando a matriz, o que pode ser feito através da exponenciação da diagonal da matriz centraldo resultado da decomposição, a qual contém os autovalores da matriz original.
<br><br>
Porém, o PageRank trabalha com uma matriz de milhões de valores, ou seja, até que o algoritmo se estabilize, existe um número imenso de iterações, ou seja de exponenciações da matriz de links. Assim, o maior autovalor passa a ser dominante na matriz, assim como a matriz de links final tem como vetor dominante um vetor associado com o maior autovalor, da mesma forma que o autovetor final.